In [46]:
import os, sys, time, random
import pandas as pd
import numpy as np
from os.path import join
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import preprocessing
from sklearn.cross_validation import StratifiedKFold

sys.path.append('/home/ymm/kaggle/xgboost_hyperopt')
import utils.bosch_functions as bosch_functions
from utils.wrapped_xgboost import xgboost_classifier
from utils.validation_tools import score_MCC, MCC, create_validation_index
from utils.models import CombinedModel
from utils.data_munge import remove_single_value_columns
from utils.feature_engineering import NumericalFeatureEngineering, getRelativeTimeColumns, BasicDate_FeatureEngineering
from utils.feature_engineering import getTimeChangeColumns, getTimeSteps, build_IndexFeatures, build_sortedData_indexDiff
from utils.feature_engineering import BasicCat_FeatureEngineering, encode_categorical_by_dep_var


data_path = '/home/ymm/bosch/'

train_num_file   = 'train_numeric.csv'
train_cat_file   = 'train_categorical.csv'
train_date_file  = 'train_date.csv'
test_num_file    = 'test_numeric.csv'
test_cat_file    = 'test_categorical.csv'
test_date_file   = 'test_date.csv'

sample_submission_file   = 'sample_submission.csv'

start_time_column_name = 'L0_S0_D1'
id_column_name = 'Id'
dep_var_name = 'Response'

In [47]:
tot_row_num = 1183747
num_rows = 100000
skip = sorted(random.sample(xrange(1,tot_row_num + 1),tot_row_num - num_rows))

In [48]:
#'''
start_time = time.time()
## randomly select certain rows
train_num = pd.read_csv(join(data_path, train_num_file),    index_col='Id', skiprows=skip, nrows=num_rows)
train_dat = pd.read_csv(join(data_path, train_date_file),   index_col='Id', skiprows=skip, nrows=num_rows)
train_cat = pd.read_csv(join(data_path, train_cat_file),    index_col='Id', skiprows=skip, nrows=num_rows)

test_num = pd.read_csv(join(data_path, test_num_file),      index_col='Id', nrows=num_rows)
test_dat = pd.read_csv(join(data_path, test_date_file),     index_col='Id', nrows=num_rows)
test_cat = pd.read_csv(join(data_path, test_cat_file),      index_col='Id', nrows=num_rows)

print 'finish loading date using {} seconds'.format(round(time.time() - start_time, 0))
#'''

finish loading date using 74.0 seconds


In [49]:
remove_single_value_columns(train_num, 'Response', test=test_num)
remove_single_value_columns(train_dat, test=test_dat)
remove_single_value_columns(train_cat, test=test_cat)

raw train data dimension:  (50000, 969)
raw test data dimension:  (50000, 968)
processed train data dimension:  (50000, 969)
processed test data dimension:  (50000, 968)
raw train data dimension:  (50000, 1156)
raw test data dimension:  (50000, 1156)
processed train data dimension:  (50000, 1146)
processed test data dimension:  (50000, 1146)
raw train data dimension:  (50000, 2140)
raw test data dimension:  (50000, 2140)
processed train data dimension:  (50000, 1307)
processed test data dimension:  (50000, 1307)


In [50]:
dat_columns = train_dat.columns.tolist()
num_columns = train_num.columns.tolist()
num_columns.remove(dep_var_name)

def build_column_dict(columns):
    station_dict = {}
    line_dict = {}
    for col in columns:
        stationList = col.split('_')[0:2]
        stationKey = ('_').join(stationList)
        lineKey = col.split('_')[0]
        
        if lineKey not in line_dict:
            line_dict[lineKey] = [col]
        else:
            line_dict[lineKey].append(col)
                    
        if stationKey not in station_dict:
            station_dict[stationKey] = [col]
        else:
            station_dict[stationKey].append(col)
            
    return station_dict, line_dict


def build_station_features(df, col_dict, prefix='dat'):
    features = pd.DataFrame()
    for key, value in col_dict.items():
        features['{}_{}_{}'.format(prefix, key, 'mean')] = df[value].mean(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'max')] = df[value].max(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'min')] = df[value].min(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'var')] = df[value].var(axis=1)
    return features


def build_station_index_features(train, test = None):
    selected_columns = []
    for col in train.columns:
        if 'mean' in col or 'var' in col:
            selected_columns.append(col)
            
    if test is not None:
        train_test = pd.concat([train[selected_columns], test[selected_columns]], axis=0)
    else:
        train_test = train[selected_columns]
        
    train_test['index'] = train_test.index
    new_fea = pd.DataFrame()
    ## function to build index based on the given columns
    build_sortedData_indexDiff(train_test, new_fea, selected_columns)
    
    return new_fea


dat_col_dict, dat_line_dict = build_column_dict(dat_columns)
num_col_dict, num_line_dict = build_column_dict(num_columns)

dat_col_dict.update(dat_line_dict)
num_col_dict.update(num_line_dict)

In [51]:
start_time = time.time()

train_dat_stations = build_station_features(train_dat, dat_col_dict, 'dat')
test_dat_stations = build_station_features(test_dat, dat_col_dict, 'dat')

train_num_stations = build_station_features(train_num, num_col_dict, 'num')
test_num_stations = build_station_features(test_num, num_col_dict, 'num')

num_station_index = build_station_index_features(train_num_stations, test_num_stations)
dat_station_index = build_station_index_features(train_dat_stations, test_dat_stations)

print 'finish feature engineering date station using {} minutes'.format(round((time.time() - start_time)/60, 2))

finish feature engineering date station using 0.91 minutes


In [52]:
combined_train_station_num = pd.concat([train_num_stations, num_station_index.ix[train_num_stations.index]], axis=1)
combined_train_station_dat = pd.concat([train_dat_stations, dat_station_index.ix[train_dat_stations.index]], axis=1)

combined_test_station_num = pd.concat([test_num_stations, num_station_index.ix[test_num_stations.index]], axis=1)
combined_test_station_dat = pd.concat([test_dat_stations, dat_station_index.ix[test_num_stations.index]], axis=1)

In [53]:
print combined_train_station_num.isnull().sum().sum(), combined_train_station_dat.isnull().sum().sum() 

7912227 7912227


In [54]:
print combined_test_station_num.isnull().sum().sum(), combined_test_station_dat.isnull().sum().sum() 

7908313 7908313


In [55]:
remove_single_value_columns(combined_train_station_num, test=combined_test_station_num)
remove_single_value_columns(combined_train_station_dat, test=combined_test_station_dat)

raw train data dimension:  (50000, 432)
raw test data dimension:  (50000, 432)
processed train data dimension:  (50000, 431)
processed test data dimension:  (50000, 431)
raw train data dimension:  (50000, 432)
raw test data dimension:  (50000, 432)
processed train data dimension:  (50000, 431)
processed test data dimension:  (50000, 431)


In [56]:
station_fillna_value = 9999999
combined_train_station_num.fillna(station_fillna_value, inplace=True)
combined_test_station_num.fillna(station_fillna_value, inplace=True)
combined_train_station_dat.fillna(station_fillna_value, inplace=True)
combined_test_station_dat.fillna(station_fillna_value, inplace=True)

In [57]:
combined_train_station_num.head()

,num_L2_S28_mean,num_L2_S28_max,num_L2_S28_min,num_L2_S28_var,num_L3_S31_mean,num_L3_S31_max,num_L3_S31_min,num_L3_S31_var,num_L2_S26_mean,num_L2_S26_max,...,num_L1_S25_var_index_diff_0,num_L1_S25_var_index_diff_1,num_L1_S24_mean_index_diff_0,num_L1_S24_mean_index_diff_1,num_L1_S24_var_index_diff_0,num_L1_S24_var_index_diff_1,num_L3_S39_mean_index_diff_0,num_L3_S39_mean_index_diff_1,num_L3_S39_var_index_diff_0,num_L3_S39_var_index_diff_1
Id,,,,,,,,,,,,,,,,,,,,,
34,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,...,1,-1,1,-1,1,-1,1,-1,1,-1
79,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,...,2,-4,2,-1,2,-1,2,-1,2,-1
93,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,...,3,-1,3,-1,3,-1,3,-1,3,-1
126,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,...,7,-2,5,-2,5,-2,5,-2,5,-2
201,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,...,2,-1,2,-1,2,-1,2,-1,2,-1


In [58]:
print combined_train_station_num.isnull().sum().sum(), combined_train_station_dat.isnull().sum().sum()

0 0


#### build categorical features

In [59]:
'''
def BasicCat_FeatureEngineering(train_cat):
    ## feature engineering on the date features
    encoder = preprocessing.LabelEncoder()
    column_names = train_cat.columns.tolist()
    column_names.append('NaN')
    encoder.fit(column_names)
    dat_new_fea = pd.DataFrame()
    dat_new_fea['cat_sum'] = train_cat.sum(axis=1)
    dat_new_fea['cat_mean'] = train_cat.mean(axis=1)
    dat_new_fea['cat_nan_count'] = train_cat.isnull().sum(axis=1)
    dat_new_fea['cat_max'] = train_cat.max(axis=1)
    dat_new_fea['cat_min'] = train_cat.min(axis=1)
    dat_new_fea['cat_max_min_diff'] = dat_new_fea['cat_max'] - dat_new_fea['cat_min']
    dat_new_fea['cat_max_min_ratio'] = dat_new_fea['cat_min'] / dat_new_fea['cat_max']

    dat_new_fea['cat_idxmax'] = train_cat.idxmax(axis=1)
    dat_new_fea['cat_idxmax'].fillna('NaN', inplace=True)
    dat_new_fea['cat_idxmax'] = encoder.transform(dat_new_fea['cat_idxmax'])
    dat_new_fea['cat_idxmin'] = train_cat.idxmin(axis=1)
    dat_new_fea['cat_idxmin'].fillna('NaN', inplace=True)
    dat_new_fea['cat_idxmin'] = encoder.transform(dat_new_fea['cat_idxmin'])
    return dat_new_fea



def encode_categorical_by_dep_var(train, test, dep_var_column='Response'):
    for col_name in train.columns:
        if col_name == dep_var_column:
            continue
        
        train[col_name] = train[col_name].astype(str)
        test[col_name] = test[col_name].astype(str)
        dep_var_mean = train[[col_name, dep_var_column]].groupby(col_name).mean()
    
        dep_var_dict = {}
        for level in dep_var_mean.index.tolist():
            dep_var_dict[level] = dep_var_mean.ix[level, dep_var_column]
    
        train[col_name] = train[col_name].replace(dep_var_dict)  
        test[col_name] = test[col_name].replace(dep_var_dict)
'''

"\ndef BasicCat_FeatureEngineering(train_cat):\n    ## feature engineering on the date features\n    encoder = preprocessing.LabelEncoder()\n    column_names = train_cat.columns.tolist()\n    column_names.append('NaN')\n    encoder.fit(column_names)\n    dat_new_fea = pd.DataFrame()\n    dat_new_fea['cat_sum'] = train_cat.sum(axis=1)\n    dat_new_fea['cat_mean'] = train_cat.mean(axis=1)\n    dat_new_fea['cat_nan_count'] = train_cat.isnull().sum(axis=1)\n    dat_new_fea['cat_max'] = train_cat.max(axis=1)\n    dat_new_fea['cat_min'] = train_cat.min(axis=1)\n    dat_new_fea['cat_max_min_diff'] = dat_new_fea['cat_max'] - dat_new_fea['cat_min']\n    dat_new_fea['cat_max_min_ratio'] = dat_new_fea['cat_min'] / dat_new_fea['cat_max']\n\n    dat_new_fea['cat_idxmax'] = train_cat.idxmax(axis=1)\n    dat_new_fea['cat_idxmax'].fillna('NaN', inplace=True)\n    dat_new_fea['cat_idxmax'] = encoder.transform(dat_new_fea['cat_idxmax'])\n    dat_new_fea['cat_idxmin'] = train_cat.idxmin(axis=1)\n    dat_

In [60]:
start_time = time.time()

train_cat['Response'] = train_num['Response']
encode_categorical_by_dep_var(train_cat, test_cat, dep_var_column='Response', fill_missing=True)
train_cat.drop('Response', axis=1, inplace=True)

print 'finish encoding categorical features using {} seconds'.format(round(time.time() - start_time, 0))

train_cat_Basics = BasicCat_FeatureEngineering(train_cat)
test_cat_Basics  = BasicCat_FeatureEngineering(test_cat)

print 'finish generating all categorical features using {} seconds'.format(round(time.time() - start_time, 0))


finish encoding categorical features using 8.0 seconds
finish generating all categorical features using 22.0 seconds


In [61]:
print train_cat.shape, test_cat.shape
train_cat.head()

(50000, 1307) (50000, 1307)


,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,L0_S2_F43,L0_S2_F45,L0_S2_F47,L0_S2_F49,L0_S2_F51,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
Id,,,,,,,,,,,,,,,,,,,,,
34,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,...,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96
79,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,...,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96
93,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,...,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96
126,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,...,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96
201,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,...,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96,5.96


In [62]:
start_time = time.time()
combined_train_cat = pd.concat([train_cat, train_cat_Basics], axis=1)
combined_test_cat  = pd.concat([test_cat, test_cat_Basics], axis=1)                                                                                                                                                 
print 'finish feature engineering date using {} seconds'.format(round((time.time() - start_time), 2))

finish feature engineering date using 0.48 seconds


In [63]:
print combined_train_cat.isnull().sum(axis=0).sum(), combined_test_cat.isnull().sum(axis=0).sum()

0 0


In [64]:
combined_test_cat.head()

,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,L0_S2_F43,L0_S2_F45,L0_S2_F47,L0_S2_F49,L0_S2_F51,...,L3_S49_F4240,cat_sum,cat_mean,cat_nan_count,cat_max,cat_min,cat_max_min_diff,cat_max_min_ratio,cat_idxmax,cat_idxmin
Id,,,,,,,,,,,,,,,,,,,,,
1,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,...,5.96,7868.758,6.020473,0,9.877,5.034,4.843,0.509669,466,1009
2,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,...,5.96,7639.305,5.844916,0,6.082,1.938,4.144,0.318645,550,512
3,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,...,5.96,7965.554,6.094533,0,10.667,5.133,5.534,0.481204,397,1248
5,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,...,5.96,7838.734,5.997501,0,8.141,5.133,3.008,0.630512,964,1248
8,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,5.962,...,5.96,7938.973,6.074195,0,10.667,5.133,5.534,0.481204,397,1248


In [65]:
combined_test_cat.isnull().sum().sum()

0

In [66]:
start_time = time.time()

#### numerical feature engineering work
train_num_Basics = NumericalFeatureEngineering(train_num)
test_num_Basics = NumericalFeatureEngineering(test_num)

missing_value= -1.5
train_num_Basics.fillna(missing_value, inplace=True)
test_num_Basics.fillna(missing_value, inplace=True)

train_num.fillna(missing_value, inplace=True)
test_num.fillna(missing_value, inplace=True)

combined_train_num = pd.concat([train_num, train_num_Basics], axis=1)
combined_test_num  = pd.concat([test_num, test_num_Basics], axis=1)
print 'finish feature engineering numercical using {} seconds'.format(round((time.time() - start_time), 2))
print 'combined train numerical feature shape: {}, combined test numerical features shape: {}'.format(combined_train_num.shape, combined_test_num.shape)

finish feature engineering numercical using 13.37 seconds
combined train numerical feature shape: (50000, 978), combined test numerical features shape: (50000, 977)


In [67]:
print combined_train_num.isnull().sum().sum(), combined_test_num.isnull().sum().sum()

0 0


### section of date features

In [68]:

## basic features from tmp_train_dat
train_dat_Basics = BasicDate_FeatureEngineering(train_dat)
test_dat_Basics  = BasicDate_FeatureEngineering(test_dat)

In [69]:
## normalized date columns
train_dat_Norm = train_dat.apply(getRelativeTimeColumns, axis=1)
test_dat_Norm  = test_dat.apply(getRelativeTimeColumns, axis=1)
## remove single-valued columns
remove_single_value_columns(train_dat_Norm, test=test_dat_Norm)

raw train data dimension:  (50000, 1146)
raw test data dimension:  (50000, 1146)
processed train data dimension:  (50000, 960)
processed test data dimension:  (50000, 960)


In [70]:
encoder = preprocessing.LabelEncoder()
column_names = train_dat.columns.tolist()
column_names.append('NaN')
encoder.fit(column_names)

LabelEncoder()

In [71]:
## TimeDiff features
train_dat_TimeDiff = train_dat.apply(getTimeChangeColumns, axis=1)
test_dat_TimeDiff  = test_dat.apply(getTimeChangeColumns, axis=1)
TimeDiff_ColumnNames = ['time_diff_start_col', 'time_diff_end_col', 'time_diff_value',
                        'time_ratio_value', 'first_time_value', 'last_time_value', 'first_date_value']

train_dat_TimeDiff.columns = TimeDiff_ColumnNames
test_dat_TimeDiff.columns = TimeDiff_ColumnNames

for column in ['time_diff_start_col', 'time_diff_end_col']:
    train_dat_TimeDiff[column].fillna('NaN', inplace=True)
    train_dat_TimeDiff[column] = encoder.transform(train_dat_TimeDiff[column])
    
    test_dat_TimeDiff[column].fillna('NaN', inplace=True)
    test_dat_TimeDiff[column] = encoder.transform(test_dat_TimeDiff[column])


In [72]:
start_time = time.time()

## section to create timeStep features
unique_value_counts = 6
timeStep_columnNames = []
column_name_columns = []
for i in xrange(unique_value_counts):
    timeStep_columnNames.extend(['time_diff_step_{}'.format(i), 'column_counts_step_{}'.format(i),
                                 'time_cost_step_{}'.format(i), 'first_column_step_{}'.format(i)])
    column_name_columns.append('first_column_step_{}'.format(i))

train_dat_TimeStep = train_dat_Norm.apply(getTimeSteps, axis=1)
test_dat_TimeStep  = test_dat_Norm.apply(getTimeSteps, axis=1)
train_dat_TimeStep.columns = timeStep_columnNames
test_dat_TimeStep.columns  = timeStep_columnNames

for column in column_name_columns:
    train_dat_TimeStep[column].fillna('NaN', inplace=True)
    test_dat_TimeStep[column].fillna('NaN', inplace=True)
    train_dat_TimeStep[column] = encoder.transform(train_dat_TimeStep[column])
    test_dat_TimeStep[column] = encoder.transform(test_dat_TimeStep[column])


print 'finish generating TimeStep features using {} minutes'.format(round((time.time() - start_time)/60, 0))


finish generating TimeStep features using 11.0 minutes


In [73]:
#print train_dat_Norm.isnull().sum().sum(), train_dat_Basics.isnull().sum().sum(), train_dat_TimeStep.isnull().sum().sum(), train_dat_TimeDiff.isnull().sum().sum()

##### fill up missing values

In [74]:
'''
tmp_train_dat = train_dat_TimeDiff.copy()
tmp_test_dat = test_dat_TimeDiff.copy()

if 'start_time' in train_dat_Basics:
    tmp_train_dat['start_time'] = train_dat_Basics['start_time']
    tmp_test_dat['start_time']  = test_dat_Basics['start_time']
'''


"\ntmp_train_dat = train_dat_TimeDiff.copy()\ntmp_test_dat = test_dat_TimeDiff.copy()\n\nif 'start_time' in train_dat_Basics:\n    tmp_train_dat['start_time'] = train_dat_Basics['start_time']\n    tmp_test_dat['start_time']  = test_dat_Basics['start_time']\n"

In [75]:
start_time = time.time()
combined_train_dat = pd.concat([train_dat_Norm, train_dat_Basics, train_dat_TimeDiff, train_dat_TimeStep], axis=1)
combined_test_dat  = pd.concat([test_dat_Norm, test_dat_Basics, test_dat_TimeDiff, test_dat_TimeStep], axis=1)                                                                                                                                                 
print 'finish feature engineering date using {} minutes'.format(round((time.time() - start_time)/60, 2))

finish feature engineering date using 0.02 minutes


In [76]:
start_time = time.time()
train_test_datIndex_features = build_IndexFeatures(combined_train_dat, combined_test_dat)
print 'finish feature engineering date index using {} minutes'.format(round((time.time() - start_time)/60, 2))

finish feature engineering date index using 0.02 minutes


#### index columns fill up missing

In [77]:
train_test_datIndex_features.fillna(-1., inplace=True)

#### after filling up missing, re-create the date columns

In [78]:
#print combined_train_dat.shape, combined_test_dat.shape
#combined_train_dat.head()

In [79]:
train_dat_Norm.fillna(-1., inplace=True)
test_dat_Norm.fillna(-1., inplace=True)

train_dat_Basics.fillna(-1., inplace=True)
test_dat_Basics.fillna(-1., inplace=True)

train_dat_TimeStep.fillna(0, inplace=True)
test_dat_TimeStep.fillna(0, inplace=True)

train_dat_TimeDiff.fillna(0, inplace=True)
test_dat_TimeDiff.fillna(0, inplace=True)

In [80]:
combined_train_dat = pd.concat([train_dat_Norm, train_dat_Basics, train_dat_TimeDiff, train_dat_TimeStep], axis=1)
combined_test_dat  = pd.concat([test_dat_Norm, test_dat_Basics, test_dat_TimeDiff, test_dat_TimeStep], axis=1)  

In [81]:
print combined_test_dat.isnull().sum().sum(), combined_train_dat.isnull().sum().sum()

0 0


In [82]:
#train_test_datIndex_features['index_ratio'].min()
#train_test_datIndex_features.isnull().sum().sum()

#### load the feature importances from xgboost models

In [83]:
xgb_feature_imp = pd.read_csv('/home/ymm/full_data_xgb_feature_importance.csv', index_col='feature')

In [84]:
xgb_feature_imp.head()

,fscore_0,norm_fscore_0,fscore_1,norm_fscore_1,fscore_2,norm_fscore_2,fscore_3,norm_fscore_3,fscore_4,norm_fscore_4,...,fscore_8,norm_fscore_8,fscore_9,norm_fscore_9,fscore_10,norm_fscore_10,fscore_11,norm_fscore_11,fscore_sum,norm_fscore_sum
feature,,,,,,,,,,,,,,,,,,,,,
first_time_value_index_diff_1,1357.0,0.013342,1480.0,0.014495,773.0,0.032962,1701.0,0.011999,1198.0,0.018810,...,1723.0,0.012308,747.0,0.031607,1671.0,0.011886,1152.0,0.018246,15033.0,0.228153
L3_S33_F3857,1317.0,0.012948,1226.0,0.012007,316.0,0.013475,1797.0,0.012676,648.0,0.010175,...,1745.0,0.012465,391.0,0.016544,1874.0,0.013330,708.0,0.011214,12276.0,0.149472
last_time_value_index_diff_1,1015.0,0.009979,1059.0,0.010372,410.0,0.017483,1310.0,0.009241,840.0,0.013189,...,1488.0,0.010629,382.0,0.016163,1329.0,0.009453,743.0,0.011768,10737.0,0.145426
L3_S33_F3859,1223.0,0.012024,1161.0,0.011371,264.0,0.011258,1722.0,0.012147,744.0,0.011682,...,1556.0,0.011115,375.0,0.015867,1649.0,0.011729,786.0,0.012449,11676.0,0.144343
last_time_value_index_diff_0,1020.0,0.010028,1087.0,0.010646,350.0,0.014925,1351.0,0.009530,707.0,0.011101,...,1391.0,0.009937,310.0,0.013117,1320.0,0.009389,612.0,0.009693,10258.0,0.132143


In [85]:
## sort by the norm_fscore_sum
sorted_combined_imp = xgb_feature_imp.sort_values(by=['norm_fscore_sum'], ascending=False)
#imp_feature = sorted_combined_imp.index[sorted_combined_imp['norm_fscore_sum'] >= 0.005].tolist()
imp_feature = sorted_combined_imp.index.tolist()

In [86]:
def select_important_features(df, imp_feature, test_df = None, dep_var_name = 'Response'):
    imp_col_names = [col for col in df.columns if col in imp_feature]
    print 'total {} columns in original DataFrame, select {} columns'.format(df.shape[1], len(imp_col_names))
    train_col_names = imp_col_names[:]
    test_col_names = imp_col_names[:]
    if dep_var_name in df.columns:    
        train_col_names.append(dep_var_name)
    if test_df is None:
        return df[train_col_names]
    else:
        return df[train_col_names], test_df[test_col_names]


In [87]:
combined_train_num, combined_test_num  = select_important_features(combined_train_num, imp_feature, combined_test_num)
combined_train_dat, combined_test_dat  = select_important_features(combined_train_dat, imp_feature, combined_test_dat)
train_test_datIndex_features = select_important_features(train_test_datIndex_features, imp_feature)

total 978 columns in original DataFrame, select 691 columns
total 1001 columns in original DataFrame, select 108 columns
total 23 columns in original DataFrame, select 22 columns


#### combine all the features together

In [88]:
## combined data with station features
#combined_train = pd.concat([train_dat_stations, train_num_stations, combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([test_dat_stations, test_num_stations, combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [89]:
#combined_train = pd.concat([combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [90]:
## combined data with categorical features
#combined_train = pd.concat([combined_train_cat, combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([combined_test_cat,  combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [91]:
## combined data with all features
combined_train = pd.concat([combined_train_station_dat, combined_train_station_num, combined_train_cat, combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
combined_test  = pd.concat([combined_test_station_dat, combined_test_station_num, combined_test_cat,  combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [92]:
print combined_train.isnull().sum().sum(), combined_test.isnull().sum().sum()

0 0


In [93]:
print combined_test.shape
combined_test.head()

(50000, 2999)


,dat_L2_S28_mean,dat_L2_S28_max,dat_L2_S28_min,dat_L2_S28_var,dat_L3_S31_mean,dat_L3_S31_max,dat_L3_S31_min,dat_L3_S31_var,dat_L2_S26_mean,dat_L2_S26_max,...,first_date_value_index_ratio_1,first_date_value_index_ratio_2,time_ratio_value_index_diff_0,time_ratio_value_index_diff_1,first_time_value_index_diff_0,first_time_value_index_diff_1,last_time_value_index_diff_0,last_time_value_index_diff_1,first_date_value_index_diff_0,first_date_value_index_diff_1
Id,,,,,,,,,,,,,,,,,,,,,
1,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.00,9999999.00,...,60973.000,94011.00,-60087,-3232,-158518,-3232,-68013,-3232,-69475,-1
2,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.00,9999999.00,...,20524.500,31769.00,-720791,-62612,-3722,-5943,-1341168,-61348,1,-1
3,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,704.11,704.11,...,20928.000,20878.00,-1962074,-24124,-1713048,-24124,-182117,-24124,1,-5
5,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,255.50,255.50,...,680.400,19458.00,-88702,-97262,-74259,-24195,-74259,-61577,-74259,-63560
8,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,760.93,760.93,...,8004.375,9348.25,-141736,-1395506,-1263631,-4313,-548728,-4313,5,-9


In [94]:
print combined_train.shape
combined_train.head()

(50000, 3000)


,dat_L2_S28_mean,dat_L2_S28_max,dat_L2_S28_min,dat_L2_S28_var,dat_L3_S31_mean,dat_L3_S31_max,dat_L3_S31_min,dat_L3_S31_var,dat_L2_S26_mean,dat_L2_S26_max,...,first_date_value_index_ratio_1,first_date_value_index_ratio_2,time_ratio_value_index_diff_0,time_ratio_value_index_diff_1,first_time_value_index_diff_0,first_time_value_index_diff_1,last_time_value_index_diff_0,last_time_value_index_diff_1,first_date_value_index_diff_0,first_date_value_index_diff_1
Id,,,,,,,,,,,,,,,,,,,,,
34,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,...,362.235294,2265.147059,-1842341,-5871,-76692,-5871,-94328,-4528,-26982,-5871
79,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,...,86.493671,53.848101,-95264,-70837,-26920,-1486988,-50259,-26920,12,-1
93,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,...,121.989247,941.301075,-21859,-71016,-131545,-4106,-1193889,-57270,-249700,-71016
126,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,...,141.253968,623.134921,-793582,-3597,-917261,-3597,-917261,-3597,-917261,-6243
201,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,...,34.751244,245.457711,-96928,-752598,-43864,-10581,-423771,-12919,-35983,-6877


#### KFold cross-validation

In [95]:
params = {}
params["eta"]                      = 0.0075
params["subsample"]                = 0.8
params["colsample_bytree"]         = 0.8
params["num_round"]                = 501
params["max_depth"]                = 5
params["gamma"]                    = 0
params["metrics"]                  = 'auc'
params['eval_metric']              = 'auc'
params["seed"]                     = 999
params['verbose_eval']             = 50
## whether to use weights
params['use_base_score']           = True
params['use_weights']              = True
#params['use_scale_pos_weight']     = True
params["val"]                      = False


In [96]:

skf = StratifiedKFold(combined_train[dep_var_name], 4)

for train_index, valid_index in skf:
    valid_data = combined_train.iloc[valid_index]
    tmp_train  = combined_train.iloc[train_index]

    y = tmp_train[dep_var_name].values
    X = tmp_train.drop(dep_var_name, axis=1)

    valid_y = valid_data[dep_var_name].values
    valid_X = valid_data.drop(dep_var_name, axis=1)
    
    model = xgboost_classifier(label_name = dep_var_name, params = params, model_file='test_bosch_xgb_model')
    model.fit(tmp_train, dep_var_name)
    
    pred = model.predict(valid_X)
    print 'result from using constant fraction: \n', score_MCC(valid_y, pred)
    print '\n'
    print 'result from using flexsible threshold:', CombinedModel.mcc_eval_func(valid_y, pred)

scale_pos_weight: 167.156950673
a base_score 0.00594682524867 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.759023
[50]	train-auc:0.943626
[100]	train-auc:0.966481
[150]	train-auc:0.982596
[200]	train-auc:0.990381
[250]	train-auc:0.993962
[300]	train-auc:0.996303
[350]	train-auc:0.997596
[400]	train-auc:0.998471
[450]	train-auc:0.998999
[500]	train-auc:0.999309
the xgboost fit is finished by using 512.0 seconds, saved into test_bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.0059995200384
threshold for preds: 0.0274208657996
0.154929985514


result from using flexsible threshold: (0.20552173685540615, 0.04819463565945625)
scale_pos_weight: 167.156950673
a base_score 0.00594682524867 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.73263
[50]	train-auc:0.935127
[100]	train-auc:0.95768
[150]	train-auc:0.973508
[200]	train-auc:0.986704
[250]	train-auc:0.992463
[300]	train-auc:0.995568
[350]	train-auc:0.997182
[400]	train-auc:0.998057
[450]	train-auc:0.998704
[500]	train-auc:0.999105
the xgboost fit is finished by using 514.0 seconds, saved into test_bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.0059995200384
threshold for preds: 0.0224218471318
0.128102366007


result from using flexsible threshold: (0.17495003632481068, 0.06440790742635727)
scale_pos_weight: 166.415178571
a base_score 0.00597317404869 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.710101
[50]	train-auc:0.929215
[100]	train-auc:0.954512
[150]	train-auc:0.974787
[200]	train-auc:0.986535
[250]	train-auc:0.991752
[300]	train-auc:0.995074
[350]	train-auc:0.996943
[400]	train-auc:0.998021
[450]	train-auc:0.998692
[500]	train-auc:0.99915
the xgboost fit is finished by using 494.0 seconds, saved into test_bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00592047363789
threshold for preds: 0.0227514473451
0.0756082440589


result from using flexsible threshold: (0.11610650229570822, 0.0710984617471695)
scale_pos_weight: 166.415178571
a base_score 0.00597317404869 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.711511
[50]	train-auc:0.93912
[100]	train-auc:0.964627
[150]	train-auc:0.980588
[200]	train-auc:0.987685
[250]	train-auc:0.991302
[300]	train-auc:0.99468
[350]	train-auc:0.996814
[400]	train-auc:0.998045
[450]	train-auc:0.998717
[500]	train-auc:0.99919
the xgboost fit is finished by using 509.0 seconds, saved into test_bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00592047363789
threshold for preds: 0.0192040979714
0.184360215346


result from using flexsible threshold: (0.26104864307720244, 0.057706885039806366)


#### regular models

In [ ]:
#print tmp_train['index_ratio'].min(), tmp_train['index_ratio'].max()

In [44]:
############## Section of regular validation #######################
train_index, valid_index = create_validation_index(combined_train, 0.3, dep_var_name, True)
valid_data = combined_train.ix[valid_index]
tmp_train  = combined_train.ix[train_index]

y = tmp_train[dep_var_name].values
X = tmp_train.drop(dep_var_name, axis=1)

valid_y = valid_data[dep_var_name].values
valid_X = valid_data.drop(dep_var_name, axis=1)


In [42]:
rf_params = {'random_state' : 9999, 'n_estimators' : 3000, 'max_depth' : 5, 'criterion' : 'gini', 'n_jobs' : -1}
et_params = {'random_state' : 9999, 'n_estimators' : 500, 'max_depth' : 5, 'criterion' : 'gini', 'n_jobs' : -1}
rf_clf = RandomForestClassifier(**rf_params)
rf_clf = rf_clf.fit(X, y)

et_clf = RandomForestClassifier(**et_params)
et_clf = et_clf.fit(X, y)

In [43]:
rf_pred = rf_clf.predict_proba(valid_X)[:, 1]
et_pred = et_clf.predict_proba(valid_X)[:, 1]

print 'result from using constant fraction: \n'
print score_MCC(valid_y, rf_pred)
print score_MCC(valid_y, et_pred)
print '\n'
print 'result from using flexsible threshold:'
print CombinedModel.mcc_eval_func(valid_y, rf_pred)
print CombinedModel.mcc_eval_func(valid_y, et_pred)


result from using constant fraction: 

mean of groud truth: 0.00593372891526
threshold for preds: 0.0579520484467
0.140970165563
mean of groud truth: 0.00593372891526
threshold for preds: 0.0660397376776
0.140970165563


result from using flexsible threshold:
(0.1933435433683039, 0.07455889273561173)
(0.1685155598098684, 0.08649280269190743)


In [45]:
model = xgboost_classifier(label_name = dep_var_name, params = params, model_file='test_bosch_xgb_model')
model.fit(tmp_train, dep_var_name)
pred = model.predict(valid_X)

print 'result from using constant fraction: \n', score_MCC(valid_y, pred)
print '\n'
print 'result from using flexsible threshold:', CombinedModel.mcc_eval_func(valid_y, pred)

scale_pos_weight: 166.468899522
a base_score 0.00597125796406 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.730501
[50]	train-auc:0.916098
[100]	train-auc:0.93997
[150]	train-auc:0.963734
[200]	train-auc:0.981598
[250]	train-auc:0.989991
[300]	train-auc:0.994931
[350]	train-auc:0.99744
[400]	train-auc:0.998508
[450]	train-auc:0.99904
[500]	train-auc:0.999385
the xgboost fit is finished by using 510.0 seconds, saved into test_bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00593372891526
threshold for preds: 0.0241102440794
0.1522731897


result from using flexsible threshold: (0.20338393874433255, 0.09097399562597275)
